In [ ]:
import astropy
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import os
from pathlib import Path
from glob import glob
import re
import logging
from rich.progress import Progress
import numpy as np
from sunpy.map import Map
import pandas as pd
from datetime import datetime, date, timedelta
import sscws
from sunpy.net import Fido, attrs as a
from functools import reduce

from matplotlib.colors import LogNorm, PowerNorm, Normalize
#from tqdm.notebook import tqdm
from tqdm import tqdm

import warnings
import cv2 # pip install opencv-python

from sunpy.map.maputils import all_coordinates_from_map
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
base_path = '/mnt/onboard_data/visualization/cme_video'
os.makedirs(base_path, exist_ok=True)

In [ ]:
fnames_a = sorted(glob("/mnt/onboard_data/data/cor2/20140222_*_n*a.fts"))
fnames_b = sorted(glob("/mnt/onboard_data/data/cor2/20140222_*_n*b.fts"))

print(len(fnames_a), len(fnames_b))

In [ ]:
warnings.simplefilter('ignore')
for f in tqdm(fnames_a):
    # plot map to jpg
    m = Map(f)
    m = Map(np.log(m.data / m.exposure_time.value), m.meta)
    #print('VALUE RANGE', m.data.min(), m.data.max())
    #print(m.exposure_time)
    plt.figure(figsize=(4, 4))
    m.plot(norm=Normalize(vmin=4, vmax=7))
    plt.savefig(os.path.join(base_path, 'cor2sa_' + os.path.basename(f).replace('.fts', '.jpg')), dpi=100)
    plt.close()

In [ ]:
f = fnames_a[1]

ref_data = []
for i in range(1, 21):
    m = Map(fnames_a[i])
    ref_data.append( m.data ) #/ m.exposure_time.value )

ref_map = np.mean(ref_data, axis=0)
ref_map

In [ ]:
plt.imshow(np.sqrt(ref_map), origin="lower", cmap="stereocor2")
plt.show()

In [ ]:
f = fnames_a[42]
print(f)

m = Map(f)

pixel_coords = all_coordinates_from_map(m)
solar_center = SkyCoord(0*u.deg, 0*u.deg, frame=m.coordinate_frame)
pixel_radii = np.sqrt((pixel_coords.Tx-solar_center.Tx)**2 +
                      (pixel_coords.Ty-solar_center.Ty)**2)
# r2 masking
mask = 1 - ((pixel_radii / pixel_radii.max()) ** 2)*0.5
mask = mask.value
mask[pixel_radii.value >= 0.9 * pixel_coords.Tx.max().value] = np.nan

In [ ]:
#m = Map(np.log(m.data / m.exposure_time.value / mask), m.meta)
#m.plot(norm=Normalize())

#data = np.log( (m.data - ref_map.data) / m.exposure_time.value / mask) # 6, 7
data = ( (m.data - ref_map)  / mask) # 20, 45 # /m.exposure_time.value
plt.imshow(data, origin="lower", cmap="stereocor2", vmin=0, vmax=300) # mirror to m.plot

plt.show()

print(np.nanmin(data), np.nanmax(data), data.min(), data.max())

In [ ]:
plt.imshow(mask)
plt.colorbar()
plt.show()

In [ ]:
m.data.min(), m.data.max(), m.exposure_time

### Generate video

https://github.com/FrontierDevelopmentLab/2023-europe-space-weather/blob/ground/sunerf/evaluation/video_cme.py

In [ ]:
base_path

In [ ]:
fnames_a

In [ ]:
warnings.simplefilter('ignore')
for f in tqdm(fnames_a):
    m = Map(f)

    pixel_coords = all_coordinates_from_map(m)
    solar_center = SkyCoord(0*u.deg, 0*u.deg, frame=m.coordinate_frame)
    pixel_radii = np.sqrt((pixel_coords.Tx-solar_center.Tx)**2 +
                        (pixel_coords.Ty-solar_center.Ty)**2)
    # r2 masking
    mask = 1 - ((pixel_radii / pixel_radii.max()) ** 2)*0.5
    mask = mask.value
    mask[pixel_radii.value >= 0.9 * pixel_coords.Tx.max().value] = np.nan

    data = ( (m.data - ref_map)  / mask) # 20, 45 # /m.exposure_time.value

    #plt.figure(figsize=(4, 4))
    plt.imshow(data, origin="lower", cmap="stereocor2", vmin=0, vmax=300) # mirror to m.plot
    plt.savefig(os.path.join(base_path, 'cor2sa_' + os.path.basename(f).replace('.fts', '.jpg')), dpi=100)
    plt.close()

In [ ]:
test_img = cv2.imread( os.path.join(base_path, "cor2sa_20140222_150915_n4c2a.jpg") )
# img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
test_img = test_img[...,::-1] # reverse red and blue channels
plt.imshow(test_img)
#plt.show()

In [ ]:
video_path = base_path

video_name = os.path.join(video_path, 'video.mp4')

images = sorted(glob(os.path.join(video_path, '*.jpg')))
frame = cv2.imread(images[0])
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, 10, (width,height))

for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()